In [17]:
# First, load all of the data files of accused witches.
# Second, clean data.

import os
import pandas as pd

# Specify the directory where are CSV files are located
folder_path = '../ArcGIS Online Files'

# Create an empty list to store DataFrames
allData = []

# List all CSV files in the specified folder which start with name 'Accused'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and filename.startswith('Accused'):
        file_path = os.path.join(folder_path, filename)

        df = pd.read_csv(file_path, encoding = "ISO-8859-1")
        
        # Change and combine columns which indicate the name category into consistent name
        df.rename(columns={'Longitude': 'Long', 'Latitude': 'Lat'}, inplace=True)
        
        # Append the DataFrame to the list
        allData.append(df)


merged_df = pd.concat(allData, ignore_index=True)

# Create another dataframe to read and store the Time of Death csv file that is queried from Wiki Data, and merges with the above dataframe.
# timeofDeath_df = pd.read_csv("TimeofDeath.csv")
# merged_df = pd.merge(merged_df, timeofDeath_df, on='Name', how='left')
merged_df.to_csv('meiged_df')
merged_df

,Wikidata Page,Name,Residence,Long,Lat,Gender,Ethnicity,Manner of Death,Occupation,Place of Detainment,Social Classification,Wikipedia Page,Place of Death
0,http://www.wikidata.org/entity/Q43390211,Jonnet McKennan,Balmurrie,-4.805504,54.959995,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q43390291,Jonet Braidheid,Balmakeith,-3.852752,57.580476,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q43390319,Wife of Soirle McAllexander,Isle of Bute,-5.056360,55.835690,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q43390500,Christian Watson,North Berwick,-2.717000,56.058000,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q43390507,Janet Conochie,Bo'ness,-3.608911,56.016811,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180,http://www.wikidata.org/entity/Q43390439,Grissell Jaffray,Dundee,-2.970000,56.464000,Female,NaN,Capital Punishment,NaN,Dundee,Middling,https://en.wikipedia.org/wiki/Grissel_Jaffray,Dundee
3181,http://www.wikidata.org/entity/Q43390306,Isobell Young,East Barns,-2.454800,55.978000,Female,NaN,Capital Punishment,NaN,Edinburgh,Middling,https://en.wikipedia.org/wiki/Issobell_Young,Castle Hill
3182,http://www.wikidata.org/entity/Q43389998,Margaret Burges,Cramond,-3.300000,55.966700,Female,NaN,Capital Punishment,NaN,NaN,Middling,https://en.wikipedia.org/wiki/Margaret_Burges,Castle Hill
3183,http://www.wikidata.org/entity/Q43393922,Margaret Fulton,Bearsden,-4.333202,55.919238,Female,NaN,Capital Punishment,NaN,Paisley,Middling,https://en.wikipedia.org/wiki/Paisley_witches,Gallows Green


## Arrows

In [16]:
# !pip install pyecharts
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType,GeoType,SymbolType

from pyecharts.globals import CurrentConfig, NotebookType

CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

line_df = pd.merge(
    merged_df[pd.notna(merged_df['Place of Detainment'])],
    merged_df[['Residence']].drop_duplicates(),
    how = 'inner',
    left_on = 'Place of Detainment', right_on = 'Residence'
    ).rename(columns = {'Residence_x' : 'Residence'})
line_df = line_df[line_df['Place of Detainment'] != line_df['Residence']]
line_df = line_df.groupby(['Residence','Place of Detainment']).agg({'Lat' : 'count'}).reset_index().rename(columns = {'Lat' : 'size'})
location = merged_df[['Residence','Lat','Long']].drop_duplicates()
location['size'] = 0.1


geo =  Geo(opts.InitOpts(
    # 图表画布宽度，css 长度单位。
    width = "1920px",

    # 图表画布高度，css 长度单位。
    height = "2000px",))


for i, row in location.iterrows():
    geo.add_coordinate(name=row.Residence,longitude=row.Long,latitude=row.Lat)

geo.add_schema(maptype="world")
# geo.add(
#          "",  # 系列名称, 可不设置
#          [(i,j) for i,j in zip(location['Residence'],location['size'])], # 数据
#          #color='#16c79a', # 标记颜色
#          type_=ChartType.EFFECT_SCATTER,     # 线条
#         #  effect_opts=opts.EffectOpts(symbol='arrow', # 类型
#         #                              symbol_size=1, # 标记大小
#         #                              color="blue",# 箭头颜色 
#         #                             ),   
#         # linestyle_opts=opts.LineStyleOpts(curve=0.2), # 设置线条弧度大小
#      )
geo.add(
        "arrows", 
        [(i,j) for i,j in zip(line_df['Residence'],line_df['Place of Detainment'])],# 数据
        type_=GeoType.LINES,      # 涟漪散点
        effect_opts=opts.EffectOpts(symbol=SymbolType.ARROW,
                                    trail_length=0.9,
                                    symbol_size=2,
                                    color="blue"# 箭头颜色
                                   ), 
        linestyle_opts=opts.LineStyleOpts(curve=0.2, type_= "dotted"),
        # color="white", # 蓝色blue  白色 white 灰色grey
        # symbol_size=20,  # 标记的大小
    )

geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False)) 
geo.set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=130000), title_opts=opts.TitleOpts(title="Geo FLow Chart - Residence to Detainment"))    # 图表标题

# geo.render_notebook()
geo.render()

## Residence Hotpot

In [13]:
import plotly.express as px
import plotly
import plotly.graph_objects as go


def density_map(df):
    fig = px.density_mapbox(
        df,
        lat='Lat',
        lon='Long',
        radius=8,  # Adjust the radius as needed
        center=dict(lat=df['Lat'].mean(), lon=df['Long'].mean()),  # Set the center of the map
        zoom=5,  # Set the initial zoom level
        mapbox_style="stamen-terrain",  # Choose a Mapbox map style
        title='Density Map of Accused Witches',
    )
    
    # Add a triangle at the center
    calibrate_lat, calibrate_lon = df['Lat'].mean(), df['Long'].mean()-4
    calibrate_coor = {'lon':calibrate_lon,'lat':calibrate_lat}
    
    fig.add_trace(go.Scattermapbox(
        mode='markers',
        lat=[calibrate_lat, calibrate_lat + 0.2, calibrate_lat - 0.2, calibrate_lat],
        lon=[calibrate_lon, calibrate_lon + 0.2, calibrate_lon - 0.2, calibrate_lon],
        marker=dict(size=10, color='red'),
        line=dict(color='red'),
        fill='toself',  # Fill the area inside the triangle
        name='Triangle'
    ))
    
    fig.show()
    plotly.offline.plot(fig, filename='residence_v2.html')
    
    return dict(lat=df['Lat'].mean(), lon=df['Long'].mean()), calibrate_coor


center_of_residence, calibrate_coor = density_map(merged_df)

## Detainment Hotpot

In [14]:
import plotly.express as px
import plotly
import plotly.graph_objects as go
import re

detention_df = pd.read_csv('detention_coor.csv')

def getLon(coor):
    pattern = re.compile(r'[0-9,.,-]+')
    return float(pattern.findall(coor)[0])

def getLat(coor):
    pattern = re.compile(r'[0-9,.,-]+')
    return float(pattern.findall(coor)[1])
    
detention_df['Long'] = detention_df['place_of_detention_coordinate'].apply(getLon)
detention_df['Lat'] = detention_df['place_of_detention_coordinate'].apply(getLat)

def density_map(df):
    fig = px.density_mapbox(
        df,
        lat='Lat',
        lon='Long',
        radius=8,  # Adjust the radius as needed
        center=center_of_residence,  # Set the center of the map
        zoom=5,  # Set the initial zoom level
        mapbox_style="stamen-terrain",  # Choose a Mapbox map style
        title='Density Map of Accused Witches',
    )
    
    calibrate_lat, calibrate_lon = calibrate_coor['lat'], calibrate_coor['lon']
    fig.add_trace(go.Scattermapbox(
        mode='markers',
        lat=[calibrate_lat, calibrate_lat + 0.2, calibrate_lat - 0.2, calibrate_lat],
        lon=[calibrate_lon, calibrate_lon + 0.2, calibrate_lon - 0.2, calibrate_lon],
        marker=dict(size=10, color='red'),
        line=dict(color='red'),
        fill='toself',  # Fill the area inside the triangle
        name='Triangle'
    ))    
    fig.show()
    plotly.offline.plot(fig, filename='detainment_v2.html')
    # fig.write_image("fig1.png")
    # fig.write_image("density_map.png")


# Call the function to draw
density_map(detention_df)